In [135]:
import pandas as pd
import requests
import re
from tqdm import tqdm
import sys

In [62]:
def create_url(baseurl, params):
    return baseurl + "&".join([f"{x}={y}" for x, y in params.items()])

In [96]:
expr = re.compile(r"\{\{Term\|BotW\|((\w* ?)+)\|link\|?\}\}")

In [84]:
baseurl = " https://zelda.fandom.com/api.php?"
params = {
    "action": "query",
    "format": "json",
    "prop": "revisions",
    "rvslots": "*",
    "rvprop": "content",
    "indexpageids": True
    }
titles = [
    "Characters_in_Breath_of_the_Wild", 
    "Enemies_in_Breath_of_the_Wild", 
    "Bosses_in_Breath_of_the_Wild"
    ]

In [112]:
# Extracting all links of format {{Term|BoTW|(here)|link}}

extraction = []
for title in titles:
    new_params = params
    new_params["titles"] = title
    url = create_url(baseurl, new_params)
    
    data = requests.get(url).json()
    pid = data["query"]["pageids"][0]
    text = data["query"]["pages"][pid]["revisions"][0]["slots"]["main"]["*"]
    
    if "==Traps==" in text:
        text = text.split("==Traps==")[0]

    extraction.extend([x[0] for x in expr.findall(text)])

In [140]:
# Sorting the extractions after whether they are traps, places, or characters.
# First two gets removed

Chars = []

for e in tqdm(extraction):
    new_params = params
    new_params["titles"] = e + "%23Breath_of_the_Wild"
    url = create_url(baseurl, new_params)

    data = requests.get(url).json()
    pid = data["query"]["pageids"][0]
    text = data["query"]["pages"][pid]["revisions"][0]["slots"]["main"]["*"]
    
    if "Infobox Place" in text:
        continue
    
    with open(f"txts/{e}.txt", "w") as f:
        f.write(str(text.encode(encoding="utf-8", errors="xmlcharrefreplace")))
        
    if "Infobox Boss" in text:
        Chars.append([e, "Boss"])
    elif "Infobox Character" in text:
        Chars.append([e, "Ally"])
    elif "Infobox Enemy" in text:
        Chars.append([e, "Enemey"])
    elif "Infobox Animal" in text:
        Chars.append([e, "Animal"])
    else:
        Chars.append([e, "Unknown"])
    


100%|██████████| 657/657 [02:48<00:00,  3.91it/s]


In [128]:
d

Empty DataFrame
Columns: []
Index: []


In [ ]:
#